# Lab: Cross-Validation

This also includes a cross validation method for classification problems, but we'll get to that another time.

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                        summarize,
                        poly)
from sklearn.model_selection import train_test_split

from functools import partial
from sklearn.model_selection import \
    (cross_validate,
    KFold,
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

# The Validation Set Approach

We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the `Auto` data set.

We use the function `train_test_split()` to split the data into training and validation sets. As there are 392 observations, we split into two equal sets of size 196 using the argument `test_size=196`. It's generally a good idea to set a random seed when performing operations like this that contain an element of randomness, so that the results obtained can be reproduced precisely at a later time.

In [3]:
Auto = load_data("Auto")
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=0)

Now we can fit a linear regression using only the observations corresponding to the training set `Auto_train`.

In [4]:
hp_mm = MS(["horsepower"])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train["mpg"]
model = sm.OLS(y_train, X_train)
results = model.fit()

We now use the `predict()` method of the `results` evaluated on the model matrix for this model created using the validation data set. We also calculate the validation MSE of our model.

In [5]:
x_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid["mpg"]
valid_pred = results.predict(x_valid)
np.mean((y_valid - valid_pred)**2)

np.float64(23.616617069669893)

Hence our estimate for the validation MSE of the linear regression fit is 23.62.

We can also estimate the validation error for higher-degree polynomial regressions. We first provide a function `evalMSE()` that takes a model string as well as a training and test set and returns the MSE on the test set.

In [6]:
def evalMSE(terms,
            response,
            train,
            test):
    
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

Let's use this function to estimate the validation MSE using linear quadratic and cubic fits. We use the `enumerate()` function here, which gives both the values and indices of objects as one iterates over a for loop.

In [8]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly("horsepower", degree)],
                       "mpg",
                       Auto_train,
                       Auto_valid)
    
MSE

array([20.75540796, 16.94510676, 16.97437833])

These error rates are 23.62, 18.76, and 18,80, respectively. If we choose a different training/validation split instead, then we can expect somewhat different errors on the validation set.

In [9]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly("horsepower", degree)],
                       "mpg",
                       Auto_train,
                       Auto_valid)
    
MSE

array([20.75540796, 16.94510676, 16.97437833])

Using this split of the observations into a training set and a validation set, we find that the validation set error rates for the models start with linear, quadratic, and cubic terms are 20.76, 16.95, and 16.97, respectively.

These results are consistent with our previous findings: a model that predicts `mpg` using a quadratic function of `horsepower` performs better than a model that involves only a linear function of `horsepower`, and there's no evidence of improvement in using a cubic function of `horsepower`.

# Cross-Validation

In theory, the cross-validation estimate can be computed for any generalized linear model. In practice, however, the simplest way to cross-validate in Python is to use `sklearn`.

This is a problem which often confronts data scientists: "I have a function to do task A, and need to feed it into something that performs task B, so that I can compute B(A(D)), where D is my data." When A and B don't naturally speak to each other, this requires the use of a **wrapper**. In the `ISLP` package, you're provided a wrapper, `skleanr_sm()` that enables us to easily use the cross-validation tools of `sklearn` with models fit by `statsmodels`.

The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model.

Here is the wrapper in action:

In [10]:
hp_model = sklearn_sm(sm.OLS,
                      MS(["horsepower"]))
X, Y = Auto.drop(columns=["mpg"]), Auto["mpg"]
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results["test_score"])
cv_err

np.float64(24.231513517929226)

The arguments to `cross_validate()` are as follows: an object with the appropriate `fit()`, `predict()`, and `score()` methods, an array of features `X` and a response `Y`. We also included an additional argument `cv` to `cross_validate()`; specifying an integer *K* results in *K*-fold cross validation. We have provided a value corresponding to the total number of observations, which results in LOOCV. The `cross_validate()` function produces a dictionary with several components; we simply want the cross-validated test score here (MSE), which is estimated to be 24.23.

We can repeat this procedure for increasingly complex polynomial fits. To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 5, computes the associated validation error, and stores it in the ith element of the vector `cv_error`. The variable `d` in the for loop corresponds to the degree of the polynomial. We begin by initializing the vector.

In [12]:
cv_error = np.zeros(5)
H = np.array(Auto["horsepower"])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV["test_score"])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03321527])

As a result, we see a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement form using higher-degree polynomials.

Above we introduced the `outer()` method of the `np.power()` function. The `outer()` method is applied to an operation that has two arguments, such as `add()`, `min()`, or `power()`. It has two arrays as arguments, and then forms a larger array where the operation is applied to each pair of elements of the two arrays.

In [22]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
print(f"Add result:\n{np.add.outer(A, B)}\n")
print(f"Power result:\n{np.power.outer(A, B)}")

Add result:
[[ 5  7]
 [ 7  9]
 [11 13]]

Power result:
[[   9   81]
 [  25  625]
 [  81 6561]]


In the CV example above, we used *K* = *n*, but of course we can also use *K* < *n*. The code is very similar to the above (and is significantly faster). Here we use `KFold()` to partition the data into *K* = 10 random groups. We use `random_state` to set a random seed and initialize a vector `cv_error` in which we will store the CV errors corresponding to the polynomial fits of degrees one to five.

In [ ]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0)

for i, d in enumerate(range(1, 6)):
    X = np.power.outer(H, np.arange(d+1)) # Remember that `H` is the `horsepower` data as an array
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv
    )
    
    cv_error[i] = np.mean(M_CV["test_score"])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13718373])

We still see little evidence that using cubic or higher-degree polynomial terms leads to a lower test error than simply using a quadratic fit.

The `cross_validate` function is flexible and can take different splitting mechanisms as an argument. For instance, one can use the `ShuffleSplit()` function to implement the validation set approach just as easily as K-fold cross-validation.

In [26]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(["mpg"], axis=1),
                         Auto["mpg"],
                         cv=validation);

results["test_score"]

array([23.61661707])

One can estimate the variability in the test error by running the following:

In [27]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(["mpg"], axis=1),
                         Auto["mpg"],
                         cv=validation)
results["test_score"].mean(), results["test_score"].std()

(np.float64(23.802232661034168), np.float64(1.4218450941091916))

Note that this standard deviation is not a valid estimate of the sampling variability of the mean test score or the individual scores, since the randomly-selected training samples overlap and hence introduce correlations. But it does give an idea of the Mont Carlo variation incurred by picking different random folds.

We will learn bootstrapping after linear regression is complete.